# 使用情境 - 1

你現在是一個作文批改老師，你想要使用所學的 Langchain 知識來幫助你批改作文，建立一個作文批改的模型，輸入一篇作文，自訂輸出格式以滿足 cell 內題目的需求。

你應該要:

    1. 定義 schema，控制模型輸出
    2. 建立 output parser
    3. 定義輸入的模板
    4. 利用 LCEL 的語法建立一個基本的 chain
    5. 將文章 input 到 chain 來產生 output

- 範例如下：
    - 輸入：
        ```python
        input_ = {
            "article": "愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。"
        }
        ```
    - 輸出
        ```python
            {
                '切題程度': 8, 
                '文筆': 7, 
                '創意': 6, 
                '通順': 9
            }
        ```

In [1]:
import warnings
from pprint import pprint
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions.base import convert_to_openai_function
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain_setup import ChatOpenAI

In [2]:
"""
======================================================
cousre practice: 
用課程先前所學，建立一個LLM模型來批改文章，並以特定框架output

給定一篇文章 `article`，讓模型給出以下四個分數：

1. 依照切題程度給予評分，分數須介於 1~10分之間
2. 依照內容給予評分，分數須介於 1~10分之間
3. 依照創意程度給予評分，分數須介於 1~10分之間
4. 依照通順程度給予評分，分數須介於 1~10分之間

===================================================
"""

input_ = {
    "article": "愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。"
}


# ======================================================
# Define schema,validation and postprocessing
# ======================================================
class Article(BaseModel):
    """批改作文的老師"""

    relevance: int = Field(description="這篇文章的內容與題目的相關程度。介於 1 到 10。")
    writing: int = Field(description="這篇文章的文字遣詞是否精練，以及其內容深度。介於 1 到 10。")
    creativity: int = Field(description="這篇文章發想內容的創意程度。介於 1 到 10。")
    coherence: int = Field(description="文章的通順程度。介於 1 到 10。")

    @root_validator()
    def validate_values(cls, values: dict) -> dict:
        for key in values.keys():
            if not (1 <= values[key] <= 10):
                warnings.warn(f"分數 {values[key]} 不在 [1,10] 內，自動調整為 1 或 10")
                values[key] = max(min(1, values[key]), 10)
        return values


# build ouput parser
output_parser = PydanticOutputFunctionsParser(pydantic_schema=Article)

# ======================================================
# Define input template
# ======================================================
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一名國文教師，你現在要批改學生的作文，題目為朋友，評分標準需為以下四項：切題、文筆、創意、通順。"),
        ("human", "你好，請問可以批改我的作文嗎。"),
        ("ai", "好的，請給我看你的文章。"),
        ("human", "這是我的文章 :\n\n{article}。"),
    ],
)

model = ChatOpenAI(temperature=0)

function_kwargs = convert_to_openai_function(Article)

openaifn_kwargs = {"functions": [function_kwargs], "function_call": {"name": "Article"}}

chain = prompt_template | model.bind(**openaifn_kwargs) | output_parser

output = chain.invoke(input_)
print(
    {
        "切題程度": output.relevance,
        "文筆": output.writing,
        "創意": output.creativity,
        "通順": output.coherence,
    }
)

{'切題程度': 8, '文筆': 7, '創意': 6, '通順': 9}


# 使用情境 - 2

請同學利用 RunnableMap 來同時處理兩種不同種類的問題，並建立兩個語言模型來回答，一個模型用來介紹景點，另一個模型則用來介紹食物，最後用 RunnablePassthrough 在保留 input 的同時並新增 output
        
    
  - 範例如下：
      - 輸入：台灣
      - 輸出：

        ```python
        outputs: {
            'attraction introduction': Attraction_descriptor(attraction='台北101', introduction='台北101是台灣最著名的地標之一，也是世界上最高的摩天大樓之一。它擁有觀景台、購物中心、餐廳和展覽館等設施，遊客可以欣賞到壯觀的城市全景。此外，每年跨年夜時，台北101的煙火秀也是台灣最受矚目的活動之一。'), 
            'food_introduction': food_descriptor(food='小籠包', introduction='小籠包是台灣的特色美食之一，它是一種以豬肉為餡料，以麵粉皮包裹而成的小型餃子。小籠包的特色在於其餡料鮮嫩多汁，皮薄餡多，每一口都能品嚐到豐富的肉汁。它通常搭配黑醋和生薑一起食用，酸甜的醋和辣味的生薑能夠提升小籠包的風味。小籠包是台灣的代表性美食之一，絕對值得一試！')
        }
        ```
    

In [29]:
from langchain.prompts import ChatPromptTemplate
from langchain_setup import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.schema.runnable import (
    RunnableLambda,
    RunnableMap,
    RunnableBranch,
    RunnablePassthrough,
)

# Scenario: Developing a Chatbot Pipeline

# Components Initialization
# Prompt
attraction_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位智能導遊機器人。請根據輸入的疑問滿足旅客的要求。"),
        ("human", "請告訴我{place}有什麼好玩的，並推薦當地的一個景點並向我介紹"),
    ],
)

# attraction_model
attraction_model = ChatOpenAI(temperature=0)


class AttractionDescriptor(BaseModel):
    """介紹景點的導遊"""

    attraction: str = Field(description="推薦的旅遊景點")
    introduction: str = Field(description="景點的簡要介紹，介紹該景點的特色")


# build ouput parser
attraction_output_parser = PydanticOutputFunctionsParser(
    pydantic_schema=AttractionDescriptor
)

attraction_function_kwargs = convert_to_openai_function(AttractionDescriptor)
attraction_openaifn_kwargs = {
    "functions": [attraction_function_kwargs],
    "function_call": {"name": "AttractionDescriptor"},
}

# Constructing the Basic Chain - Prompt | Model | Output Parser
attraction_chain = (
    attraction_prompt
    | attraction_model.bind(**attraction_openaifn_kwargs)
    | attraction_output_parser
)

# Creating RunnableLambda instances for the functions

food_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位智能導遊機器人。請根據輸入的疑問滿足旅客的要求。"),
        ("human", "請告訴我{place}有什麼特色美食，簡要的介紹一個特色美食給我"),
    ],
)

# food_model
food_model = ChatOpenAI(temperature=0)


class FoodDescriptor(BaseModel):
    """介紹食物的導遊"""

    food: str = Field(description="美食名稱")
    introduction: str = Field(description="美食的簡要介紹")


# build ouput parser
food_output_parser = PydanticOutputFunctionsParser(pydantic_schema=FoodDescriptor)


food_function_kwargs = convert_to_openai_function(FoodDescriptor)
food_openaifn_kwargs = {
    "functions": [food_function_kwargs],
    "function_call": {"name": "FoodDescriptor"},
}
# Exercise:

# Combining Runnable Functions using RunnableLambda
food_chain = food_prompt | food_model.bind(**food_openaifn_kwargs) | food_output_parser

# Combine different model output through RunnableMap and RunnablePassthrough

branched_runnable = RunnableMap({
    "attraction introduction": attraction_chain,
    "food_introduction": food_chain
})

inputs = {"place": "台灣"}
add_by_pass = RunnablePassthrough.assign(output= branched_runnable)
outputs = add_by_pass.invoke(inputs)
print(f'input: {outputs["place"]}\noutputs: {outputs["output"]}')

input: 台灣
outputs: {'attraction introduction': Attraction_descriptor(attraction='台北101', introduction='台北101是台灣最著名的地標之一，也是世界上最高的摩天大樓之一。它擁有觀景台、購物中心、餐廳和展覽館等設施，遊客可以欣賞到壯觀的城市全景。此外，每年跨年夜時，台北101的煙火秀也是台灣最受矚目的活動之一。'), 'food_introduction': food_descriptor(food='小籠包', introduction='小籠包是台灣的特色美食之一，它是一種以豬肉為餡料，以麵粉皮包裹而成的小型餃子。小籠包的特色在於其餡料鮮嫩多汁，皮薄餡多，每一口都能品嚐到豐富的肉汁。它通常搭配黑醋和生薑一起食用，酸甜的醋和辣味的生薑能夠提升小籠包的風味。小籠包是台灣的代表性美食之一，絕對值得一試！')}
